<a href="https://colab.research.google.com/github/coraldx5/generativeai_intro_book/blob/master/chap05_LLM_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch transformers fugashi ipadic

In [ ]:
import transformers
from transformers import BertJapaneseTokenizer, BertForMaskedLM
from transformers import pipeline

# トークナイザと訓練済みモデルの読み込み
model = BertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
# パイプラインの定義
fill_mask = pipeline('fill-mask',
                    model=model, #← 言語モデルの指定
                    tokenizer=tokenizer, #← トークナイザの指定
                    top_k=6 #← 表示する候補数の指定
                    )
# 結果を表示する関数の定義
def predictmask(text):
  print('---'*10)
  print(f'元の文章：「{text}」')
  print(f'[MASK]部の候補：')
  for res in fill_mask(text):
    print(f"{res['score']:.4f}: {res['token_str']}")

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
predictmask('サングラスをかけた[MASK]が公園を駆け回る。')
predictmask('サングラスをかけた[MASK]を食べるのが楽しみだ')
predictmask('生卵をかけた[MASK]を食べるのが楽しみだ')

------------------------------
元の文章：「サングラスをかけた[MASK]が公園を駆け回る。」
[MASK]部の候補：
0.2395: 男
0.0848: 少年
0.0511: 少女
0.0433: 犬
0.0331: 青年
0.0293: 女性
------------------------------
元の文章：「サングラスをかけた[MASK]を食べるのが楽しみだ」
[MASK]部の候補：
0.0882: もの
0.0608: ケーキ
0.0493: 料理
0.0314: 朝食
0.0312: 犬
0.0305: ご飯
------------------------------
元の文章：「生卵をかけた[MASK]を食べるのが楽しみだ」
[MASK]部の候補：
0.1659: もの
0.1010: ご飯
0.0815: 料理
0.0752: スープ
0.0658: 卵
0.0490: パン


In [ ]:
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

# Load pre-trained tokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

# Tokenize input
text = 'サングラスをかけた少女が公園を駆け回る'
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 5
tokenized_text[masked_index] = '[MASK]'
# ['テレビ', 'で', '[MASK]', 'の', '試合', 'を', '見る', '。']

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# [571, 12, 4, 5, 608, 11, 2867, 8]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
# tensor([[ 571,   12,    4,    5,  608,   11, 2867,    8]])

# Load pre-trained model
model = BertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
model.eval()

# Predict
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0][0, masked_index].topk(10) # 予測結果の上位5件を抽出

# Show results
for i, index_t in enumerate(predictions.indices):
    index = index_t.item()
    token = tokenizer.convert_ids_to_tokens([index])[0]
    print(i, token)

['サン', '##グラス', 'を', 'かけ', 'た', '少女', 'が', '公園', 'を', '駆け', '##回る']


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0 男
1 女性
2 男女
3 男性
4 子供
5 若者
6 人物
7 猫
8 人
9 カップル
